# Model Inference

In [8]:
import os, sys
import numpy as np
from nltk.tokenize import TweetTokenizer
from enum import Enum
import preprocessor as p
import torch

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.model.BiLSTMClassifier import EnhancedBiLSTM
from src.utils import label_to_emoji

In [9]:
class Model(Enum):
    BiLSTM_GLOVE_100D = 3
    BiLSTM_GLOVE_200D = 4
    BiLSTM_FASTTEXT_300D = 5

In [42]:
embedding_matrix_glove = np.load("../data/embeddings/embedding_matrix_glove_twitter_100.npy")
vocab = np.load("../data/embeddings/vocab.npy", allow_pickle=True).item()

vocab_size, embedding_dim_glove = embedding_matrix_glove.shape

model_glove = EnhancedBiLSTM(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim_glove,
    hidden_dim=256,
    num_classes=20,
    embedding_matrix=embedding_matrix_glove,
    freeze_embeddings=True,
    num_layers=2,
    dropout=0.5
)

embedding_matrix_glove_200d = np.load("../data/embeddings/embedding_matrix_glove_twitter_200.npy")
embedding_dim_glove_200d = embedding_matrix_glove_200d.shape[1]

model_glove_200d = EnhancedBiLSTM(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim_glove_200d,
    hidden_dim=256,
    num_classes=20,
    embedding_matrix=embedding_matrix_glove_200d,
    freeze_embeddings=True,
    num_layers=2,
    dropout=0.5
)

embedding_matrix_fasttext = np.load("../data/embeddings/embedding_matrix_crawl_subword_300.npy")
embedding_dim_fasttext = embedding_matrix_fasttext.shape[1]

model_fasttext = EnhancedBiLSTM(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim_fasttext,
    hidden_dim=128,
    num_classes=20,
    embedding_matrix=embedding_matrix_fasttext,
    freeze_embeddings=True,
    num_layers=2,
    dropout=0.5
)

In [46]:
def encode(text, vocab, max_len=50):
    text = p.tokenize(text)
    tokens = TweetTokenizer().tokenize(text.lower())
    ids = [vocab.get(t, vocab["<UNK>"]) for t in tokens]
    if len(ids) < max_len:
        ids = ids + [vocab["<PAD>"]] * (max_len - len(ids))
    else:
        ids = ids[:max_len]
    return ids

def predict_emojis(text_ids, execute_model):
    execute_model.eval()
    with torch.no_grad():
        X = torch.tensor([text_ids], dtype=torch.long)
        logits = execute_model(X)
        preds = logits.softmax(dim=1)[0].cpu().numpy()

        preds = {label_to_emoji(i): float(preds[i]) for i in range(len(preds))}

        return preds

In [47]:

def infer(model: Model, text: str, top_k: int = 5):
    execute_model = None

    text_ids = encode(text, vocab)
    result = None

    if model == Model.BiLSTM_GLOVE_100D:
        execute_model = model_glove
        execute_model.load_state_dict(torch.load(f"models/bilstm_glove_twitter_best.pth", map_location=torch.device('cpu')))
        result = predict_emojis(text_ids, execute_model)
    elif model == Model.BiLSTM_GLOVE_200D:
        execute_model = model_glove_200d
        execute_model.load_state_dict(torch.load(f"models/bilstm_glove_twitter_200d_best.pth", map_location=torch.device('cpu')))
        result = predict_emojis(text_ids, execute_model)
    elif model == Model.BiLSTM_FASTTEXT_300D:
        execute_model = model_fasttext
        execute_model.load_state_dict(torch.load(f"models/bilstm_fasttext_best.pth", map_location=torch.device('cpu')))
        result = predict_emojis(text_ids, execute_model)
    else:
        raise ValueError("Unsupported model type")
    
    sorted_emojis = sorted(result.items(), key=lambda x: x[1], reverse=True)
    return sorted_emojis[:top_k]

In [53]:
tweet = "I love programming! #coding"

predictions = infer(Model.BiLSTM_FASTTEXT_300D, tweet)

print(f"Predictions using {Model.BiLSTM_FASTTEXT_300D.name}:")
print(predictions)

Predictions using BiLSTM_FASTTEXT_300D:
[('❤', 0.4537658095359802), ('😍', 0.19293206930160522), ('💕', 0.061540428549051285), ('💙', 0.0509660542011261), ('😊', 0.04383263736963272)]
